#   <center>Machine Learning Engineer Nanodegree</center>
##   <center>Project: Dogs vs. Cats Redux: Kernels Edition</center>
<center>
Author: Kyle Chen<br>
Date: 20180506<br>
Version: 20180506v1
</center>

---

###   写在前面
-   这次的毕业项目选做猫狗, 一方面是因为资料比较多, 资源比较丰富. 另外一方面, 图形识别也是当下的一大热点. 虽然毕业后可能会继续从事传统机器学习方面的研究, 但是能够有这段经历还是很不错的, 也能丰富自己的简历与深度学习的知识.
-   在这个项目中, 将通过评估几种模型与不同程度的调优, 来不断优化我们的模型框架, 以达到最终的top 10%标准.

###   准备数据
-   本文中的DataSet是已经从kaggle上拖取到本地, 并放入当前工作目录中, 但是由于Github的大小限制, 如果你想要执行并研究其中的一些代码, 请自行准备好数据集, 并存放到./DataSet目录下. kaggle传送门 <a href='https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data'>dogs-vs-cats-redux-kernels-edition/data</a>.
-   或者你可以直接运行下框的代码创建目录结构与拖取数据.

In [ ]:
# 如果你已经拖取了数据, 请勿执行此代码框中的代码
!mkdir DataSet
!wget -c "https://storage.googleapis.com/kaggle-competitions-data/kaggle/5441/train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1525872567&Signature=Vp1XJm2t%2FeDE9TqhWRHhhDuCD7GmOw4LhuU9cVU%2FNKbur08UKSw8UkDRm%2B6quFq0NL41vn%2BA45YkXvwlmiyM%2Br51%2BvXpWUHtYi3XAMwxjEVn7HI7dwyEP2tSO1H9SS%2Bi5YM8e94zNQ5mrpUypxL52HDBH0BVJBGs40RFR7uAiSeLizUNwArPl5zyP11EkOPrcFC2umd8e5BmfxpfRWUNTwr4%2FpfN6AAu%2BOsPU3QakCnzqxYQ1idOsQ4AO4AecseLYtEdeXJaov0lwaUVh9BIRMZibW4Sylh9RW0QmTspNbeCdZ%2BiTzMMfHxII5DhuXznZcpHOLRIG7%2F8XqOULoMzAA%3D%3D" -O DataSet/train.zip
!wget -c "https://storage.googleapis.com/kaggle-competitions-data/kaggle/5441/test.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1525872601&Signature=aHXH%2B1qiQOk5ny9b3YrivSZQB23neGcHxVubwp6olX8SPz6V5wfkqpbs2ncy%2B%2BLozRBLx%2BG86KdjqsuGuo%2FbYYjMwwh%2F1784dKFaNlFxBR3x8jIn6ji221MWwCkX9Cij20xC9ECpaXWBap8jYypRlAp%2Ff8AlTJF1zY8xQ84su8Gs2y8tDVs9Gt0OuiKu4dNJ017ZPjclPYNjm2%2BCG1GcpgCmZy6qkqvW%2FsuMPr%2BLcGFB1X0xrqYLxmX1JJGlikoZ%2FjQiJ5ZYjIhnLm05BhWdegChS24hnDyF4Mo4DoI9r9NBpRIPqF3kW%2BSZ0ci%2FRgutEvqr7OXcuRpOIR4pPYVZdw%3D%3D" -O DataSet/test.zip
!unzip DataSet/train.zip -d DataSet/
!unzip DataSet/test.zip -d DataSet/

###   关于DataSet
-   先了解下数据集的组成.

In [1]:
!ls -ahl DataSet/

total 816M
drwxr-xr-x 4 root root 4.0K May  9 12:53 .
drwxr-xr-x 6 root root 4.0K May  9 12:52 ..
drwxr-xr-x 2 root root 276K May  8 14:56 test
-rw-r--r-- 1 root root 272M May  8 14:56 test.zip
drwxr-xr-x 2 root root 740K May  8 14:56 train
-rw-r--r-- 1 root root 544M May  8 14:56 train.zip


-   test目录下存放的是kaggle准备好的测试集.
-   train目录下存放的是训练集, 当然, 还需要将其细分为训练集与验证集两个部分.

In [2]:
!ls -ahl DataSet/train/ | head -n 5

total 595M
drwxr-xr-x 2 root root 740K May  8 14:56 .
drwxr-xr-x 4 root root 4.0K May  9 12:53 ..
-rw-r--r-- 1 root root  13K May  8 14:56 cat.0.jpg
-rw-r--r-- 1 root root  22K May  8 14:56 cat.10000.jpg
ls: write error: Broken pipe


-   发现数据集命名是有规则的, 遵循label.n.jpg的原则.

In [3]:
!echo "cats | $(find DataSet/train/ -name 'cat*' | wc -l)"
!echo "dogs | $(find DataSet/train/ -name 'dog*' | wc -l)"

cats | 12500
dogs | 12500


-   发现cats/dogs样本类型分布均匀.

###   Import Libs

In [64]:
# 导入我们后面需要用到的库
import os
import re
import h5py
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.utils import shuffle
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Input, Dropout, Flatten, Dense, BatchNormalization
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.applications.xception import Xception
from keras.preprocessing.image import ImageDataGenerator

###   Initial Global Variables

In [32]:
# initial global variables
TRAIN_DIR = 'DataSet/train'
TEST_DIR = 'DataSet/test'
TRANSFER_DIR = 'DataSet/transfer'
TRANSFER_TRAIN_DIR = 'DataSet/transfer/train'
TRANSFER_TEST_DIR = 'DataSet/transfer/test'
np.random.seed(2018)

###   导入数据集

In [6]:
# load data from TRAIN_DIR
def load_data(width, height, channels):
    img_list = os.listdir(TRAIN_DIR)
    nums = len(img_list)
    data = np.empty((nums, width, height, channels), dtype="float32")
    label = np.empty((nums, ))
    
    i = 0
    while i < nums:
        img = img_list[i]
        imgObj = Image.open("{}/{}".format(TRAIN_DIR, img))
        arr = np.asarray(imgObj, dtype="float32")
        arr.resize((width, height, channels))
        data[i, :, :, :] = arr
        
        if re.match(r'^cat\.', img) != None:
            label[i] = 0
            
        elif re.match(r'^dog\.', img) != None:
            label[i] = 1            
        i += 1
    return(data, label)

###   CNN模型框架
-   Load数据

In [7]:
# run load data
data, label = load_data(224, 224, 3)
data, label = shuffle(data, label)

-   CNN模型设计

In [8]:
cnn_model = Sequential()
shape_input = (len(data[0]), len(data[0][0]), len(data[0][0][0]))
cnn_model.add(Conv2D(filters=16, kernel_size=2, input_shape=shape_input))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=2, padding='valid'))
cnn_model.add(Dense(133, activation='relu'))
cnn_model.add(Conv2D(filters=32, kernel_size=2))
cnn_model.add(MaxPooling2D(pool_size=2, padding='valid'))
cnn_model.add(Dense(133, activation='relu'))
cnn_model.add(Conv2D(filters=64, kernel_size=2))
cnn_model.add(MaxPooling2D(pool_size=2, padding='valid'))
cnn_model.add(GlobalAveragePooling2D(dim_ordering='default'))
cnn_model.add(Dense(1, activation='sigmoid'))
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 223, 223, 16)      208       
_________________________________________________________________
batch_normalization_1 (Batch (None, 223, 223, 16)      64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
dense_1 (Dense)              (None, 111, 111, 133)     2261      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 110, 110, 32)      17056     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 55, 55, 32)        0         
_________________________________________________________________
dense_2 (Dense)              (None, 55, 55, 133)       4389      
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `GlobalAveragePooling2D` call to the Keras 2 API: `GlobalAveragePooling2D(data_format=None)`
  if sys.path[0] == '':


-   编译模型

In [9]:
# 编译模型
# 在keras中, 提供binary_crossentropy函数, 就是我们需要的LogLoss
cnn_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

-   训练CNN模型

In [10]:
# 这里在8c 16g的mbpr上跑了差不多两个半小时
epochs = 10
batch_size = 20
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.cnn.hdf5', 
                                   verbose=1, save_best_only=True)
cnn_model.fit(data, label, validation_split = 0.3,
                epochs = epochs, batch_size = batch_size,
                callbacks=[checkpointer], verbose=1)

Train on 17500 samples, validate on 7500 samples
Epoch 1/10
17500/17500 [==============================] - 40s 2ms/step - loss: 0.6759 - acc: 0.5766 - val_loss: 0.6637 - val_acc: 0.6247

Epoch 00001: val_loss improved from inf to 0.66375, saving model to saved_models/weights.best.cnn.hdf5
Epoch 2/10
17500/17500 [==============================] - 36s 2ms/step - loss: 0.6552 - acc: 0.6119 - val_loss: 0.6497 - val_acc: 0.6189

Epoch 00002: val_loss improved from 0.66375 to 0.64973, saving model to saved_models/weights.best.cnn.hdf5
Epoch 3/10
17500/17500 [==============================] - 37s 2ms/step - loss: 0.6472 - acc: 0.6186 - val_loss: 0.6542 - val_acc: 0.5907

Epoch 00003: val_loss did not improve from 0.64973
Epoch 4/10
17500/17500 [==============================] - 36s 2ms/step - loss: 0.6408 - acc: 0.6305 - val_loss: 0.6380 - val_acc: 0.6341

Epoch 00004: val_loss improved from 0.64973 to 0.63801, saving model to saved_models/weights.best.cnn.hdf5
Epoch 5/10
17500/17500 [=======

-   这里可以发现, 用CNN优化之后, 还是在___. LogLoss没有太大的提升.

###   使用Xception
-   将train下的数据分类至transfer目录下

In [26]:
# 将train下的数据分类并拷贝至transfer/{cat,dog}目录下
!mkdir DataSet/transfer/train/cat -p
!mkdir DataSet/transfer/train/dog
!mkdir DataSet/transfer/test/pic -p
!find DataSet/train -name 'cat.*' -exec cp -rf {} DataSet/transfer/train/cat/ \;
!find DataSet/train -name 'dog.*' -exec cp -rf {} DataSet/transfer/train/dog/ \;
!find DataSet/test -name '*.jpg' -exec cp -rf {} DataSet/transfer/test/pic/ \;

-   加载数据

In [33]:
# run load data, change size to Xception default (299,299)
img_size = (299, 299)
gen = ImageDataGenerator()
X_train_gen = gen.flow_from_directory(TRANSFER_TRAIN_DIR, img_size, shuffle = False, 
                                        batch_size = 16)
X_test_gen = gen.flow_from_directory(TRANSFER_TEST_DIR, img_size, shuffle = False, 
                                        batch_size = 16, classes = None)

Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


-   训练模型设计

In [34]:
Xception_base = Xception(input_tensor = Input((img_size[0], img_size[1], 3)), 
                             weights = 'imagenet', include_top = False)
Xception_model = Model(Xception_base.input, GlobalAveragePooling2D()(Xception_base.output))
Xception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

-   利用Xception训练特征向量

In [35]:
X_train = Xception_model.predict_generator(X_train_gen)
X_test = Xception_model.predict_generator(X_test_gen)

-   保存特征权重

In [36]:
with h5py.File('saved_models/weights.Xception.hdf5') as fp:
        fp.create_dataset('train', data = X_train)
        fp.create_dataset('test', data = X_test)
        fp.create_dataset('label', data = X_train_gen.classes)

-   导入我们刚刚训练好的特征向量

In [74]:
X_train = []
X_test = []
with h5py.File('saved_models/weights.Xception.hdf5', 'r') as fp:
    X_train.append(np.array(fp['train']))
    X_test.append(np.array(fp['test']))
    y_train = np.array(fp['label'])
    
X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)
X_train, y_train = shuffle(X_train, y_train)

-   使用Xception训练好的特征向量构建模型

In [75]:
input_tensor = Input(X_train.shape[1:])
Xception_model = Model(input_tensor, Dropout(0.5)(input_tensor))
Xception_model = Model(Xception_model.input, Dense(1, activation = 'sigmoid')(Xception_model.output))
Xception_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 2048)              0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 2049      
Total params: 2,049
Trainable params: 2,049
Non-trainable params: 0
_________________________________________________________________


-   编译模型

In [76]:
Xception_model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

-   使用Xception训练模型

In [77]:
epochs = 1000
batch_size = 128
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.Xception.hdf5', 
                                   verbose=1, save_best_only=True)
Xception_model.fit(X_train, y_train, validation_split = 0.3,
                    epochs = epochs, batch_size = batch_size,
                    callbacks=[checkpointer])

Train on 17500 samples, validate on 7500 samples
Epoch 1/1000
17500/17500 [==============================] - 2s 118us/step - loss: 3.5681 - acc: 0.5451 - val_loss: 4.4661 - val_acc: 0.5371

Epoch 00001: val_loss improved from inf to 4.46606, saving model to saved_models/weights.best.Xception.hdf5
Epoch 2/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.9007 - acc: 0.5756 - val_loss: 1.3732 - val_acc: 0.6312

Epoch 00002: val_loss improved from 4.46606 to 1.37317, saving model to saved_models/weights.best.Xception.hdf5
Epoch 3/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.7340 - acc: 0.5866 - val_loss: 1.2351 - val_acc: 0.6620

Epoch 00003: val_loss improved from 1.37317 to 1.23509, saving model to saved_models/weights.best.Xception.hdf5
Epoch 4/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.6378 - acc: 0.5869 - val_loss: 1.9585 - val_acc: 0.5707

Epoch 00004: val_loss did not improve from 1.23509
Epoch 5


Epoch 00041: val_loss did not improve from 0.94869
Epoch 42/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.0650 - acc: 0.6027 - val_loss: 1.1526 - val_acc: 0.6256

Epoch 00042: val_loss did not improve from 0.94869
Epoch 43/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.1049 - acc: 0.5961 - val_loss: 2.6308 - val_acc: 0.5281

Epoch 00043: val_loss did not improve from 0.94869
Epoch 44/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.0780 - acc: 0.6006 - val_loss: 0.9263 - val_acc: 0.6487

Epoch 00044: val_loss improved from 0.94869 to 0.92633, saving model to saved_models/weights.best.Xception.hdf5
Epoch 45/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.0225 - acc: 0.6006 - val_loss: 1.9349 - val_acc: 0.6051

Epoch 00045: val_loss did not improve from 0.92633
Epoch 46/1000
17500/17500 [==============================] - 1s 31us/step - loss: 2.0573 - acc: 0.5971 - val_loss: 3.31

17500/17500 [==============================] - 1s 31us/step - loss: 2.1694 - acc: 0.5948 - val_loss: 3.2221 - val_acc: 0.5192

Epoch 00083: val_loss did not improve from 0.80664
Epoch 84/1000
17500/17500 [==============================] - 1s 31us/step - loss: 2.1235 - acc: 0.5985 - val_loss: 2.0244 - val_acc: 0.5643

Epoch 00084: val_loss did not improve from 0.80664
Epoch 85/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.1123 - acc: 0.5968 - val_loss: 1.1342 - val_acc: 0.6689

Epoch 00085: val_loss did not improve from 0.80664
Epoch 86/1000
17500/17500 [==============================] - 1s 32us/step - loss: 1.9620 - acc: 0.6091 - val_loss: 2.9077 - val_acc: 0.5792

Epoch 00086: val_loss did not improve from 0.80664
Epoch 87/1000
17500/17500 [==============================] - 1s 31us/step - loss: 2.0463 - acc: 0.6057 - val_loss: 2.3805 - val_acc: 0.5376

Epoch 00087: val_loss did not improve from 0.80664
Epoch 88/1000
17500/17500 [===========================


Epoch 00125: val_loss did not improve from 0.80664
Epoch 126/1000
17500/17500 [==============================] - 1s 31us/step - loss: 2.0510 - acc: 0.6089 - val_loss: 3.2790 - val_acc: 0.5477

Epoch 00126: val_loss did not improve from 0.80664
Epoch 127/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.1675 - acc: 0.5995 - val_loss: 2.1207 - val_acc: 0.6081

Epoch 00127: val_loss did not improve from 0.80664
Epoch 128/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.0995 - acc: 0.6043 - val_loss: 1.8681 - val_acc: 0.5776

Epoch 00128: val_loss did not improve from 0.80664
Epoch 129/1000
17500/17500 [==============================] - 1s 31us/step - loss: 2.1184 - acc: 0.6035 - val_loss: 1.8150 - val_acc: 0.6147

Epoch 00129: val_loss did not improve from 0.80664
Epoch 130/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.1288 - acc: 0.5977 - val_loss: 1.1697 - val_acc: 0.6439

Epoch 00130: val_loss did not impr

17500/17500 [==============================] - 1s 32us/step - loss: 2.1153 - acc: 0.6057 - val_loss: 0.9005 - val_acc: 0.6735

Epoch 00168: val_loss did not improve from 0.80664
Epoch 169/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.1469 - acc: 0.6018 - val_loss: 0.8168 - val_acc: 0.6875

Epoch 00169: val_loss did not improve from 0.80664
Epoch 170/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.0300 - acc: 0.6073 - val_loss: 1.1066 - val_acc: 0.6291

Epoch 00170: val_loss did not improve from 0.80664
Epoch 171/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.0647 - acc: 0.6067 - val_loss: 2.5929 - val_acc: 0.5815

Epoch 00171: val_loss did not improve from 0.80664
Epoch 172/1000
17500/17500 [==============================] - 1s 32us/step - loss: 2.1490 - acc: 0.6033 - val_loss: 2.5956 - val_acc: 0.5899

Epoch 00172: val_loss did not improve from 0.80664
Epoch 173/1000
17500/17500 [======================

...

17500/17500 [==============================] - 1s 35us/step - loss: 2.0490 - acc: 0.6126 - val_loss: 1.0658 - val_acc: 0.6759

Epoch 00805: val_loss did not improve from 0.78561
Epoch 806/1000
17500/17500 [==============================] - 1s 35us/step - loss: 2.0372 - acc: 0.6081 - val_loss: 3.0043 - val_acc: 0.5644

Epoch 00806: val_loss did not improve from 0.78561
Epoch 807/1000
17500/17500 [==============================] - 1s 35us/step - loss: 2.0276 - acc: 0.6139 - val_loss: 0.8581 - val_acc: 0.6804

Epoch 00807: val_loss did not improve from 0.78561
Epoch 808/1000
17500/17500 [==============================] - 1s 35us/step - loss: 2.0509 - acc: 0.6073 - val_loss: 2.7569 - val_acc: 0.5256

Epoch 00808: val_loss did not improve from 0.78561
Epoch 809/1000
17500/17500 [==============================] - 1s 35us/step - loss: 2.0643 - acc: 0.6088 - val_loss: 1.4320 - val_acc: 0.6059

Epoch 00809: val_loss did not improve from 0.78561
Epoch 810/1000
17500/17500 [======================


Epoch 00847: val_loss did not improve from 0.78561
Epoch 848/1000
17500/17500 [==============================] - 1s 34us/step - loss: 2.2378 - acc: 0.6014 - val_loss: 0.9728 - val_acc: 0.6712

Epoch 00848: val_loss did not improve from 0.78561
Epoch 849/1000
17500/17500 [==============================] - 1s 35us/step - loss: 2.0520 - acc: 0.6128 - val_loss: 1.8230 - val_acc: 0.5785

Epoch 00849: val_loss did not improve from 0.78561
Epoch 850/1000
17500/17500 [==============================] - 1s 35us/step - loss: 2.0798 - acc: 0.6113 - val_loss: 1.0752 - val_acc: 0.6587

Epoch 00850: val_loss did not improve from 0.78561
Epoch 851/1000
17500/17500 [==============================] - 1s 35us/step - loss: 2.1185 - acc: 0.6079 - val_loss: 3.3423 - val_acc: 0.5559

Epoch 00851: val_loss did not improve from 0.78561
Epoch 852/1000
17500/17500 [==============================] - 1s 35us/step - loss: 2.1589 - acc: 0.6036 - val_loss: 2.8755 - val_acc: 0.5333

Epoch 00852: val_loss did not impr

17500/17500 [==============================] - 1s 35us/step - loss: 2.0973 - acc: 0.6097 - val_loss: 2.4954 - val_acc: 0.5868

Epoch 00890: val_loss did not improve from 0.78561
Epoch 891/1000
17500/17500 [==============================] - 1s 35us/step - loss: 2.1307 - acc: 0.6045 - val_loss: 2.7186 - val_acc: 0.5881

Epoch 00891: val_loss did not improve from 0.78561
Epoch 892/1000
17500/17500 [==============================] - 1s 34us/step - loss: 2.0796 - acc: 0.6063 - val_loss: 2.6745 - val_acc: 0.5399

Epoch 00892: val_loss did not improve from 0.78561
Epoch 893/1000
17500/17500 [==============================] - 1s 35us/step - loss: 2.0787 - acc: 0.6092 - val_loss: 1.0138 - val_acc: 0.6581

Epoch 00893: val_loss did not improve from 0.78561
Epoch 894/1000
17500/17500 [==============================] - 1s 34us/step - loss: 2.1285 - acc: 0.6005 - val_loss: 1.8353 - val_acc: 0.6040

Epoch 00894: val_loss did not improve from 0.78561
Epoch 895/1000
17500/17500 [======================


Epoch 00932: val_loss did not improve from 0.78561
Epoch 933/1000
17500/17500 [==============================] - 1s 34us/step - loss: 2.1247 - acc: 0.6050 - val_loss: 1.6283 - val_acc: 0.5819

Epoch 00933: val_loss did not improve from 0.78561
Epoch 934/1000
17500/17500 [==============================] - 1s 35us/step - loss: 2.0168 - acc: 0.6078 - val_loss: 1.6599 - val_acc: 0.6273

Epoch 00934: val_loss did not improve from 0.78561
Epoch 935/1000
17500/17500 [==============================] - 1s 34us/step - loss: 2.0251 - acc: 0.6094 - val_loss: 2.2739 - val_acc: 0.5457

Epoch 00935: val_loss did not improve from 0.78561
Epoch 936/1000
17500/17500 [==============================] - 1s 34us/step - loss: 2.1641 - acc: 0.6025 - val_loss: 1.3812 - val_acc: 0.6151

Epoch 00936: val_loss did not improve from 0.78561
Epoch 937/1000
17500/17500 [==============================] - 1s 35us/step - loss: 1.9324 - acc: 0.6170 - val_loss: 2.2103 - val_acc: 0.5920

Epoch 00937: val_loss did not impr

17500/17500 [==============================] - 1s 34us/step - loss: 2.1093 - acc: 0.6027 - val_loss: 2.3786 - val_acc: 0.5415

Epoch 00975: val_loss did not improve from 0.78561
Epoch 976/1000
17500/17500 [==============================] - 1s 34us/step - loss: 2.0299 - acc: 0.6157 - val_loss: 2.5860 - val_acc: 0.5933

Epoch 00976: val_loss did not improve from 0.78561
Epoch 977/1000
17500/17500 [==============================] - 1s 35us/step - loss: 2.1235 - acc: 0.6121 - val_loss: 4.2076 - val_acc: 0.5368

Epoch 00977: val_loss did not improve from 0.78561
Epoch 978/1000
17500/17500 [==============================] - 1s 34us/step - loss: 2.1341 - acc: 0.6105 - val_loss: 1.7263 - val_acc: 0.6137

Epoch 00978: val_loss did not improve from 0.78561
Epoch 979/1000
17500/17500 [==============================] - 1s 34us/step - loss: 2.1782 - acc: 0.6066 - val_loss: 1.0869 - val_acc: 0.6689

Epoch 00979: val_loss did not improve from 0.78561
Epoch 980/1000
17500/17500 [======================

-   可以发现, 使用Xception比直接使用CNN的表现要好很多.
###   尝试往kaggle上提交


-   对测试集进行预测

In [ ]:
pred = Xception_model.predict(X_test)
pred = pred.clip(min = 0.005, max = 0.995)